In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualisation
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-aug-2021/train.csv')

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.loss.unique()

## Objective: 
To do EDA on the given data, and find the features that have higher impact on the target "loss" function.

## Strategy:
Use both visual and inbuilt function and select the features to do elaborate EDA. 

## Tactics:
Start with splitting the data into 25 columns to search for features. Use Heatmap to begin with. Once located, use scatter matrix to further exploration in detail.

In [ ]:
#Pop the target "loss" column into 'target' df
target = train.pop('loss')
target

In [ ]:
col25= range(1,25)
col50= range(26,50)
col75= range(51,75)
col101= range(76,101)
train.iloc[:,col101]

In [ ]:
#calculate correlation values for 25 columns with target
corr25 = train.iloc[:,col25].corrwith(target)
corr50 = train.iloc[:,col50].corrwith(target)
corr75 = train.iloc[:,col75].corrwith(target)
corr101 = train.iloc[:,col101].corrwith(target)
corr101

In [ ]:
## Let us consider the features with correalation more than 0.02 in either direction
plt.figure(figsize=(14,14))
X = corr25.index
Y = corr25.values

X50 = corr50.index
Y50 = corr50.values

X75 = corr75.index
Y75 = corr75.values

X101 = corr101.index
Y101 = corr101.values

mask1 = Y <= -0.02   # masking with respect to correlation value
mask2 = Y > 0.02
mask3 = Y > 0.0

plt.subplot(4,1,1)
plt.bar(X, Y, color = 'red')
plt.title('Target Correlation')
plt.xlabel('Features', fontsize=24, fontname="Comic Sans MS")
plt.ylabel('Correlation', fontsize=24, fontname="Comic Sans MS" )
plt.grid()
plt.xticks(corr25.index, rotation='vertical')
plt.rcParams['figure.figsize']=(20,6)
plt.rcParams['font.size'] = 14
plt.show()

plt.subplot(4,1,2)
plt.bar(X50, Y50, color = 'red')
plt.title('Target Correlation')
plt.xlabel('Features', fontsize=24, fontname="Comic Sans MS")
plt.ylabel('Correlation', fontsize=24, fontname="Comic Sans MS" )
plt.grid()
plt.xticks(corr50.index, rotation='vertical')
plt.rcParams['figure.figsize']=(20,6)
plt.rcParams['font.size'] = 14
plt.show()

plt.subplot(4,1,3)
plt.bar(X75, Y75, color = 'red')
plt.title('Target Correlation')
plt.xlabel('Features', fontsize=24, fontname="Comic Sans MS")
plt.ylabel('Correlation', fontsize=24, fontname="Comic Sans MS" )
plt.grid()
plt.xticks(corr75.index, rotation='vertical')
plt.rcParams['figure.figsize']=(20,6)
plt.rcParams['font.size'] = 14
plt.show()

plt.subplot(4,1,4)
plt.bar(X101, Y101, color = 'red')
plt.title('Target Correlation')
plt.xlabel('Features', fontsize=24, fontname="Comic Sans MS")
plt.ylabel('Correlation', fontsize=24, fontname="Comic Sans MS" )
plt.grid()
plt.xticks(corr101.index, rotation='vertical')
plt.rcParams['figure.figsize']=(20,6)
plt.rcParams['font.size'] = 14
plt.show()

In [ ]:
#f13,f25,f28,f46,f52,f53,f58,f66,f70,f99 are the 10 columns that are significant
sig_col=[14,26,29,47,53,54,59,67,71,99]

In [ ]:
#Significant columns are as follows
train.iloc[:,sig_col]

## Mutual information is another more specific method to locate the significant features. Let us try that 

In [ ]:
from sklearn.feature_selection import mutual_info_regression
#helps to find the features that are more important to the target value prediction/ calculation
def make_mi_scores(X, y):
    X = X.copy()
    for colname in X.select_dtypes(["object", "category"]):
        X[colname], _ = X[colname].factorize()
    # All discrete features should now have integer dtypes
    discrete_features = [pd.api.types.is_integer_dtype(t) for t in X.dtypes]
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features, random_state=0)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


In [ ]:
col10= range(1,10)

## The notebook crashed when all the 25,000 rows were ingested initially, so using only 20,000 rows for calculating the mutual information for every 25 columns. Out of 250,000 rows, the decision is based on very minimal data.

In [ ]:
mi_scores_25 = make_mi_scores(train.iloc[:20000,col25], target[:20000])
mi_scores_25[::-1]  # show a few features with their MI scores
## Taking f11 

In [ ]:
mi_scores_50 = make_mi_scores(train.iloc[:20000,col50], target[:20000])
mi_scores_50[::-1]  # show a few features with their MI scores
## Taking f27

In [ ]:
mi_scores_75 = make_mi_scores(train.iloc[:20000,col75], target[:20000])
mi_scores_75[::-1]  # show a few features with their MI scores
## Taking f55 

In [ ]:
mi_scores_101 = make_mi_scores(train.iloc[:20000,col101], target[:20000])
mi_scores_101[::-1]  # show a few features with their MI scores
## Taking f80

In [ ]:
# After mutual information the significant columns are 
sig_col=[12,14,26,28,29,47,53,54,56,59,67,71,81,99]

In [ ]:
#pair plot with significant columns
sns.pairplot(data=train.iloc[:,sig_col])

In [ ]:
sns.displot(target)
#samples with higher losses are less in number, so I can use it for doing feature engineering

In [ ]:
def loss_level(n):
    if n < 10:
        return 1
    elif n >=10 and n < 20:
        return 2
    elif n >= 20 and n < 30:
        return 3
    else:
        return 4
        
train['losslevel'] = target.apply(lambda x: loss_level(x))

In [ ]:
train.losslevel.value_counts().plot(kind='bar')